# Knitting

Two types of stitches:
* knit stich (legs faced)
* purl stich (head faced)

Each cell contains 2 dependant knots: to legs of northener stich, head of southerner stich.

Any neighbour combinations are possible.


In [1]:
from random import random
import numpy as np
import sympy as sp
from ipywidgets import widgets
import k3d

from myutils import *
from mycurves import *

t, s, u, v, r, a, b, c, d = sp.symbols("t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P0 P1 P2 P3')

In [2]:
from matplotlib.cm import get_cmap
palette = hexpalette(get_cmap('tab10'))

In [3]:
south = vector(0, -1, 0)
north = vector(0, 1, 0)
west = vector(-1, 0, 0)
east = vector(1, 0, 0)

# Energy model

Energy model by [Choi and Lo, 2003]

* $t \in [0,1]$ for half loop, $t = 0.5$ in the middle of leg
* $w$ — loop width
* $c + 2e$ — total loop height with overlapping
* $c$ — distance between knots 
* $t_h$ — fabric thickness
* $d$ — yarn diameter
* $a$ — fuckery

$x(t) = a t^3 - 1.5 a t^2 + 0.5 (a + w) t$

$y(t) = 0.5(c + 2 e)(1 - cos (\pi t))$

$z(t) = 0.5(t_h - d)(1 - cos (2\pi t))$

* $\chi d$ — horizontal loop separation (distance in knot), $1 ≤ \chi ≤ \frac{w}{2d} - 1$
* $t = \beta_1, \beta_2 $ — righmost and leftmost points of knot $x(\beta_1)  - x(\beta_2) = \chi d$, symmetric: $\beta_1 + \beta_2 = 1$
* $t = \beta_3, \beta_4 $ — intersection points $x(\beta_4) = x(\beta_3), y(\beta_4) - y(\beta_3) = c$

$\beta_{1,2} = \frac{1}{2}\left(1 ± \sqrt{\frac{a - 2w}{3a}}\right)$

$e = \frac{c}{2}\left(\frac{1}{cos \pi \beta_1} - 1\right)$

$t_h = d \left( \frac{2}{cos 2 \pi \beta_3 - cos 2 \pi \beta_4} + 1\right)$

In [17]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("a, w, c, e, z")
Q = vector(
    Ka * t**3 - 3 * half * Ka * t**2 + half*(Ka+Kw)*t,
    half * (Kc + 2*Ke)*(1 - sp.cos(sp.pi * t)),
    half * (Kz) * (1 - sp.cos(2*sp.pi*t))
)


In [18]:
# b1, b2, b3, b4 = sp.symbols("b_1, b_2, b_3, b_4")

In [19]:
# edge points
sp.solve(sp.diff(Q[0], t), t)

[(3*a - sqrt(3)*sqrt(a*(a - 2*w)))/(6*a),
 (3*a + sqrt(3)*sqrt(a*(a - 2*w)))/(6*a)]

In [27]:
controls = {
    'cell': widgets.Checkbox(description="cell", value=True),
    'a': widgets.FloatSlider(description="a", value=5, min=0, max=10.0),
    'w': widgets.FloatSlider(description="w", value=1.0, min=0, max=2.0),
    'c': widgets.FloatSlider(description="c", value=1.0, min=0, max=2.0),    
    'e': widgets.FloatSlider(description="e", value=0.5, min=0, max=1.0, step=0.01),
    'z': widgets.FloatSlider(description="z", value=0.25, min=0, max=2.0)
}
plot = k3d.plot(grid_visible=False)
plot += k3d.vectors(*make_grid((-1,1,1),(-1,1,1)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-0.5,0.5,1),(-0.5,0.5,1)), color=0x000000, use_head=False, line_width=0.01)

widgets.GridBox([
    widgets.VBox(list(controls.values())),
    plot], 
    layout=widgets.Layout(grid_template_columns="auto auto"))

GridBox(children=(VBox(children=(Checkbox(value=True, description='cell'), FloatSlider(value=5.0, description=…

In [28]:
plot_curve1 = k3d.line(vertices=[], color=palette[1], name="head")
plot += plot_curve1
plot_curve2 = k3d.line(vertices=[], color=palette[2], name="legs")
plot += plot_curve2
plot_points1 = k3d.points(positions=[], color=palette[0], opacity=0.5, point_size=0.1, shader='3d')
plot += plot_points1

In [29]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    snap = np.array(vector(-0.5*params['w'], - params['e'], 0))
    
    Q_ = Q.subs(params)
    spline0 = eval_curve(Q_, t, (0, 1, 36))
    # mirror/rotate
    #spline0 = np.concatenate([spline0, np.flip(spline0, 0) * np.array(vector(-1, -1, 1)) + np.array(vector(0.5*params['w'], 2*params['c'], 0))]) 
        
        
    if params['cell']:
        spline0 = spline0 + snap
        spline1 = np.concatenate([spline0, np.flip(spline0,0) * np.array(vector(-1, 1, 1))])  + np.array(vector(0, - params['c'], 0))
        spline2 = np.concatenate([spline0, np.flip(spline0,0) * np.array(vector(-1, 1, 1))])
    else:
        spline1 = spline0 + np.array(vector(0, - params['c'], 0))
        spline2 = spline0
    plot_curve1.vertices = spline1.astype(np.float32)
    plot_curve2.vertices = spline2.astype(np.float32)
    
#     a, w = params['a'], params['w']
#     t1 = (3*a - np.sqrt(3)*np.sqrt(a*(a - 2*w)))/(6*a)
#     t2 = (3*a + np.sqrt(3)*np.sqrt(a*(a - 2*w)))/(6*a)
#     edges = eval(Q_, {t: [t1, t2]})
#     if params['snap']:
#         edges = edges + snap
#     points1.positions = edges
    

In [30]:
redraw()

In [31]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')

# Spline model

Quadratic uniform spline.

Control points:
* rectangular polygon matching loop dimentions


In [63]:
Ka, Kw, Kc, Ke, Kz = sp.symbols("Ka, Kw, Kc, Ke, Kz")

#### hexagonal 4*2 segments
cpoints6 = [
    vector(-0.5-Ka,-Kc-1,-Kz),
    vector(-0.5+Ka,-Kc-1,-Kz),
    vector(-0.5+Kw,-1+Ke,+Kz),
    vector( -Kw,-Ke,+Kz),
    vector( -Ka, Kc,-Kz),
    vector( +Ka, Kc,-Kz),
    vector( +Kw,-Ke,+Kz),
    vector(+0.5-Kw,-1+Ke,+Kz),
    vector(+0.5-Ka,-Kc-1,-Kz),
    vector(+0.5+Ka,-Kc-1,-Kz),
]

#### triangular 3*2 segments
cpoints3 = [
    vector(-0.5-Kw,-0.5-Ke,+Kz),
    vector(-0.5,-Kc-1,-Kz),
    vector(-0.5+Kw,-0.5-Ke,+Kz),
    vector( -Kw,-0.5+Ke,+Kz),
    vector(  0, Kc,-Kz),
    vector( +Kw,-0.5+Ke,+Kz),
    vector(+0.5-Kw,-0.5-Ke,+Kz),
    vector(+0.5,-Kc-1,-Kz),
    vector(+0.5+Kw,-0.5-Ke,+Kz),
]

#### rectangular, 4*2 segments
cpoints4 = [
    vector(-0.5-Kw, -0.5-Kc, -Kz),
    vector(-0.5+Kw, -0.5-Kc, -Kz),
    vector(-0.5+Kw, -0.5-Ke, +Kz),
    vector(-Kw, -0.5+Ke, +Kz),
    vector(-Kw, -0.5+Kc, -Kz),
    vector(+Kw, -0.5+Kc, -Kz),
    vector(+Kw, -0.5+Ke, +Kz),
    vector(0.5-Kw, -0.5-Ke, +Kz),
    vector(0.5-Kw, -0.5-Kc, -Kz),
    vector(0.5+Kw, -0.5-Kc, -Kz),
]

In [84]:
controls = {
    'model': widgets.RadioButtons(description="model", value='triangular', options=('hexagonal', 'rectangular', 'triangular')),
    'plot': widgets.RadioButtons(description="plot", value='spline', options=('polygon', 'spline', 'pipe')),
    'cell': widgets.Checkbox(description="cell", value=True),
#     'Ka': widgets.FloatSlider(description="Ka", value=0.3, min=0, max=1.0, step=0.01, continuous_update=False),
    'r': widgets.FloatSlider(description="r", value=0.1, min=0, max=0.125, step=0.005, continuous_update=False),
    'auto': widgets.Checkbox(description="auto", value=False),
    'Ka': widgets.FloatSlider(description="Ka", value=0.5, min=0, max=1.0, step=0.01, continuous_update=False),    
    'Kw': widgets.FloatSlider(description="Kw", value=0.5, min=0, max=1.0, step=0.01, continuous_update=False),
    'Kc': widgets.FloatSlider(description="Kc", value=0.5, min=0, max=1.0, step=0.01, continuous_update=False),
    'Ke': widgets.FloatSlider(description="Ke", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
    'Kz': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
}
plot = k3d.plot(grid_visible=False)
plot += k3d.vectors(*make_grid((-2,2,1),(-2,2,1)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-1.5,1.5,1),(-1.5,1.5,1)), color=0x000000, use_head=False, line_width=0.01)

widgets.GridBox([widgets.VBox(list(controls.values())), plot], layout=widgets.Layout(grid_template_columns="auto auto"))

GridBox(children=(VBox(children=(RadioButtons(description='model', index=2, options=('hexagonal', 'rectangular…

In [85]:
plot_points1 = k3d.points(positions=[], color=palette[0], point_size=0.05)
plot += plot_points1
plot_lines1 = k3d.line(vertices=[], color=palette[1])
plot += plot_lines1
plot_lines2 = k3d.line(vertices=[], color=palette[2])
plot += plot_lines2
plot_pipe1 = k3d.mesh(vertices=[], indices=[], color=palette[1], flat_shading=False)
plot += plot_pipe1
plot_pipe2 = k3d.mesh(vertices=[], indices=[], color=palette[2], flat_shading=False)
plot += plot_pipe2

In [102]:
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    
    if params['auto']:
        params['Ke'] = 0.5
        params['Kw'] = (params['r']-0.01) * (0.5 - 0.37)/.12 + 0.37
        params['Kc'] = (params['r']-0.01) * (0.5 - 0.13)/.12 + 0.13
        params['Kz'] = (params['r']-0.01) * (0.3 - 0.03)/.12 + 0.03
        controls['Ke'].value = params['Ke']
        controls['Kw'].value = params['Kw']
        controls['Kc'].value = params['Kc']
        controls['Kz'].value = params['Kz']
    
    model_type = params['model']
    if model_type == 'triangular':
        cpoints = cpoints3
    elif model_type == 'rectangular':
        cpoints = cpoints4
    else:
        cpoints = cpoints6    
    splines = make_splines(u, B2, cpoints)
    
    points0 = np.array([np.array(p.subs(params), dtype=np.float) for p in cpoints])
    plot_points1.positions = points0
    
    
    plot_type = params['plot']
    
    if plot_type == 'spline':
        spline0 = np.concatenate([eval_curve(spl.subs(params), u) for spl in splines])
        spline1 = np.concatenate([spline0, np.flip(spline0, 0) * npvector(-1, 1, 1)])
        spline2 = spline1 + npvector(0, 1.0, 0)
        plot_lines1.vertices = spline1.astype(np.float32)
        plot_lines2.vertices = spline2.astype(np.float32)
    elif plot_type == 'pipe':
        pipes = [Pipe((u,v), spl.subs(params), Circle(v, params['r'])) for spl in splines]
        total = 0
        points = []
        indices = []
        for pipe in pipes:
            pnts, ids, _ = eval_pipe(pipe, (u,v), (12, 24))
            points.extend(pnts)
            indices.extend(ids + total)
            total += len(pnts)
        plot_pipe1.vertices = points
        plot_pipe1.indices = indices
        plot_pipe2.vertices = points + npvector(0, 1.0, 0).T
        plot_pipe2.indices = indices
    else: # polygon
        polygon1 = points0
        polygon2 = points0 + npvector(0, 1.0, 0)
        plot_lines1.vertices = polygon1.astype(np.float32)
        plot_lines2.vertices = polygon2.astype(np.float32)      
        plot_pipe1.visible = False
        plot_pipe2.visible = False       

    if plot_type == 'pipe':
        plot_lines1.visible = False
        plot_lines2.visible = False
        plot_pipe1.visible = True
        plot_pipe2.visible = True        
        
    else:
        plot_pipe1.visible = False
        plot_pipe2.visible = False       
        plot_lines1.visible = True
        plot_lines2.visible = True      
    


In [100]:
redraw()

In [88]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')

# Topology

Defining anchor or approximated points

Main hypotesis: shape can be determined based only on local neighbours.

## Hexagonal

From ["3D Modeling of Plain Weft Knitted Structures of Compressible Yarn", Kyosev et al, 2010] 

Anchors:
* $A^{NW}, A^{NE}$ — intersection points near the tip of a loop head
* $A^{SW}, A^{SE}$ — intersection points at sides of a loop head

Symmetrical variations $z+$ and $z-$ for above/below yarn.

Anchored splines:
* head: $south(A^{NW}), A^{SW}, A^{NW}, A^{NE}, A^{SE}, south(A^{NE})$
* w leg: $west(A^{SE}), A^{SW}, A^{NW}, north(A^{SW})$
* e leg: $east(A^{SW}), A^{SE}, A^{NE}, north(A^{SE})$

Cells preserve type of head knots, legs points are fully dependant

In [4]:
base4 = {
    'nw': np.array([-0.125, 0.25, -0.5]),
    'ne': np.array([+0.125, 0.25, -0.5]),
    'sw': np.array([-0.375, -0.25, +0.5]),
    'se': np.array([+0.375, -0.25, +0.5]),
}

def topology4(pattern, x, y):
    # generate points for splines inside single cell
    k = pattern[y][x]
    if k is None:
        return None
    ks = pattern[y+1][x]
    if ks is None:
        ks = k
    kn = pattern[y-1][x]
    if kn is None:
        kn = k
    kw = pattern[y][x-1]
    if kw is None:
        kw = k
    ke = pattern[y][x+1]
    if ke is None:
        ke = k
    
    orient = np.array([1, 1, k])
    bump = np.array([0, 0, k])
    
    points = {
            'hnw': base4['nw'] * orient,
            'hne': base4['ne'] * orient,
            'hsw': base4['sw'] * orient,
            'hse': base4['se'] * orient,
#             'snw': base['nw'] - vector(0, 1, 0),
#             'sne': base['ne'] - vector(0, 1, 0),
    }
    
    if k != kn:
        points['hnw'] -= bump * 0.5
        points['hne'] -= bump * 0.5
        points['hsw'] -= bump * 0.5
        points['hse'] -= bump * 0.5 
        
    if k != kw:
        points['hnw'] += bump * 0.5
        points['hsw'] += bump * 0.5

    if k != ke:
        points['hne'] += bump * 0.5
        points['hse'] += bump * 0.5
    
    points['lnw'] = head['nw'] + bump
    points['lne'] = head['ne'] + bump
    points['lsw'] = head['sw'] - bump
    points['lse'] = head['se'] - bump

    return points

## Triangular

Anchors:
* $A^{N}$ — tip of head
* $A^{SW}, A^{SE}$ — middles of legs
* $A^{W}, A^{E}$ — tips of counter-heads
* $A^{NW}, A^{NE}$ — middles of counter-legs

Topo dependencies:
* $A^{N}, A^{W}$ ← W
* $A^{N}, A^{E}$ ← E
* $A^{SW}, A^{SE}$ ← S
* $A^{N}, A^{NW}, A^{NE}$ ← N
   


In [5]:
def topology3(pattern, x, y):
    # generate points for splines inside single cell
    k = pattern[y][x]
    if k is None:
        return None
    ks = pattern[y+1][x]
    if ks is None:
        ks = k
    kn = pattern[y-1][x]
    if kn is None:
        kn = k
    kw = pattern[y][x-1]
    if kw is None:
        kw = k
    ke = pattern[y][x+1]
    if ke is None:
        ke = k
    
    z = -k
    
    shift = np.array([0.0, 0.0, -z])
    hshift = np.array([0.0, 0.0, -z * 0.5])
    
    points = {
        'n': np.array([0.0, 0.0, z]),
        'sw': np.array([-0.25, -0.5, -z]),
        'se': np.array([+0.25, -0.5, -z]),
        'w': np.array([-0.5, 0.0, z]),
        'e': np.array([+0.5, 0.0, z]),
        'nw': np.array([-0.25, +0.5, -z]),
        'ne': np.array([+0.25, +0.5, -z]),
    }
    
    if kw != k:
        points['n'] += hshift
        points['w'] += shift
    
    if ke != k:
        points['n'] += hshift
        points['e'] += shift
        
    if ks != k:
        points['sw'] -= hshift
        points['se'] -= hshift
        
    if kn != k:
        points['n'] -= hshift
        points['nw'] -= hshift
        points['ne'] -= hshift
        
    return points

In [6]:
chk = lambda: widgets.Checkbox(value=True, indent=False, layout=widgets.Layout(width='32px'))
brd = widgets.VBox([
    widgets.HBox([chk(), chk(), chk()]), # N
    widgets.HBox([chk(), chk(), chk()]), # M
    widgets.HBox([chk(), chk(), chk()])  # S
])

def read_pattern():
    return [[int(box.value)*2-1 for box in hbox.children] for hbox in brd.children]

controls = {
    'Kw': widgets.FloatSlider(description="Kw", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
    'Kc': widgets.FloatSlider(description="Kc", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
    'Ke': widgets.FloatSlider(description="Ke", value=0.1, min=-1.0, max=1.0, step=0.01, continuous_update=False),    
    'Kz': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
    'noise': widgets.FloatSlider(description="Kz", value=0.25, min=0, max=1.0, step=0.01, continuous_update=False),
    'topo': widgets.RadioButtons(description="topo", value='triangular', options=('hexagonal', 'triangular')),
    'mode': widgets.RadioButtons(description="mode", value='spline', options=('polygon', 'spline', 'pipe'))
}
plot = k3d.plot(grid_visible=False)

widgets.GridBox([widgets.VBox([brd, *controls.values()]), plot], layout=widgets.Layout(grid_template_columns="auto auto"))


GridBox(children=(VBox(children=(VBox(children=(HBox(children=(Checkbox(value=True, indent=False, layout=Layou…

In [7]:
plot += k3d.vectors(*make_grid((-2,2,1),(-2,2,1)), color=0x808080, use_head=False, line_width=0.005)
plot += k3d.vectors(*make_grid((-1.5,1.5,1),(-1.5,1.5,1)), color=0x000000, use_head=False, line_width=0.01)

/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/qmax/.local/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [8]:
plot_points = k3d.points(positions=[], shader='3d', opacity=0.5, point_size=0.05, color=palette[0])
plot += plot_points
plot_row1 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[1])
plot += plot_row1
plot_row2 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[2])
plot += plot_row2
plot_row3 = k3d.line(vertices=[], opacity=0.5, point_size=0.1, color=palette[3])
plot += plot_row3

In [16]:
plot_pipe2 = k3d.mesh(vertices=[], indices=[], color=0x808080, opacity=0.5)
plot += plot_pipe2

In [9]:
npsouth = np.array([0, -1, 0])

In [10]:
def make_course4(headrow, legsrow, params):
    adj = np.array([0.0625, 0, 0])    
    if legsrow is None:
        legsrow = headrow
    return np.concatenate([
        np.array([
            legsrow[x]['lsw'] + adj + npsouth,
            legsrow[x]['lnw'] + adj + npsouth,
            headrow[x]['hsw'] - adj,
            headrow[x]['hnw'] - adj,
            headrow[x]['hne'] + adj,
            headrow[x]['hse'] + adj,
            legsrow[x]['lne'] - adj + npsouth,
            legsrow[x]['lse'] - adj + npsouth,
        ]) + np.array([x, 0, 0])  for x in range(3)])    

In [11]:
from random import random

def make_course3(headrow, legsrow, params):
    adj = np.array([0, 0.125, 0])
    if legsrow is None:
        legsrow = headrow
    w = params['Kw']
    c = params['Kc']
    e = params['Ke']
    z = params['Kz']
    return np.concatenate([
        np.array([
            npsouth + [-0.5+w, -c, z*legsrow[x]['w'][2]],
            npsouth + [-0.5+w, +e, z*legsrow[x]['nw'][2]],
            [-w, -e, z*headrow[x]['sw'][2]],
            [-w, +c, z*headrow[x]['n'][2]],
            [+w, +c, z*headrow[x]['n'][2]],
            [+w, -e, z*headrow[x]['se'][2]],
            npsouth + [+0.5-w, +e, z*legsrow[x]['ne'][2]],
            npsouth + [+0.5-w, -c, z*legsrow[x]['e'][2]],
        ]) + np.array([x, 0, 0])  for x in range(3)])    

In [103]:
def redraw():
    params = {k: w.value for k, w in controls.items()}
    pattern = read_pattern()
    pattern = [[None,None,None,None,None]] + [[None] + row + [None] for row in pattern] + [[None,None,None,None,None]]
    
    if params['topo'] == 'doubleknot':
        topology = topology2
        make_course = make_course2
    elif params['topo'] == 'triangular':
        topology = topology3
        make_course = make_course3
    else:
        topology = topology4
        make_course = make_course4
        
    cells = [[topology(pattern, x, y) for x in range(1,4)] for y in range(1,4)]
    
    plot_points.positions = np.array(list(cells[1][1].values())) * np.array([1, 1, 0.5 * params['Kz']])
        
    if params['mode'] == 'spline':
        plot_row1.visible = True
        plot_row2.visible = True
        plot_row3.visible = True
        plot_pipe2.visible = False
        plot_row1.vertices = eval_splines(B2, make_course(cells[0], cells[1], params) * np.array([1, 1, .5]) + np.array([-1, +1, 0]))
        plot_row2.vertices = eval_splines(B2, make_course(cells[1], cells[2], params) * np.array([1, 1, .5]) + np.array([-1,  0, 0]))
        plot_row3.vertices = eval_splines(B2, make_course(cells[2], None, params) * np.array([1, 1, .5]) + np.array([-1, -1, 0]))
    else:
        plot_row1.vertices = make_course(cells[0], cells[1], params) * np.array([1, 1, .5]) + np.array([-1, +1, 0])
        plot_row2.vertices = make_course(cells[1], cells[2], params) * np.array([1, 1, .5]) + np.array([-1,  0, 0])
        plot_row3.vertices = make_course(cells[2], None, params) * np.array([1, 1, .5]) + np.array([-1, -1, 0])
    

In [13]:
redraw()

In [14]:
for w in controls.values():
    w.observe(lambda *args: redraw(), 'value')
for r in brd.children:
    for w in r.children:
        w.observe(lambda *args: redraw(), 'value')

# Geometry

keypoints → control points

### Parameters
* yarn thickness
* wale width
* course width

### Constraints
* minimize curvature
* minimal distance is > thickness
  * distance in Z at intersection points
  * distance in XY from middle of knot
* loop curve fits desired dimentions

### visual solution
* $r \in [0.01, 0.13]$
* $e = 0.5$
* $w \in [0.37, 0.5]$
* $c \in [0.13, 0.5]$
* $z \in [0.03, 0.3]$
